<a href="https://colab.research.google.com/github/charan2909/Quiz/blob/main/Nueral_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import dataset from data.csv file
dataset = pd.read_csv('/content/training.csv',delimiter='\t')
dataset = dataset.dropna(subset=["Temperature"])
dataset=dataset.reset_index(drop=True)
training_set = dataset.iloc[:,1:2].values.reshape(-1,1)

#Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)


x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

In [6]:
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout,Bidirectional
# Fitting RNN to training set using Keras Callbacks. Read Keras callbacks docs for more info.

In [ ]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=50,batch_size=32 )

Epoch 1/50
95/95 [==============================] - 21s 84ms/step - loss: 0.0318 - acc: 0.2505
Epoch 2/50
95/95 [==============================] - 8s 80ms/step - loss: 0.0156 - acc: 0.2406
Epoch 3/50
95/95 [==============================] - 7s 73ms/step - loss: 0.0132 - acc: 0.2551
Epoch 4/50
95/95 [==============================] - 8s 84ms/step - loss: 0.0121 - acc: 0.2436
Epoch 5/50
95/95 [==============================] - 7s 72ms/step - loss: 0.0110 - acc: 0.2587
Epoch 6/50
95/95 [==============================] - 8s 82ms/step - loss: 0.0103 - acc: 0.2558
Epoch 7/50
95/95 [==============================] - 7s 73ms/step - loss: 0.0097 - acc: 0.2459
Epoch 8/50
95/95 [==============================] - 9s 91ms/step - loss: 0.0090 - acc: 0.2459
Epoch 9/50
58/95 [=================>............] - ETA: 2s - loss: 0.0091 - acc: 0.2495

In [ ]:
# read test dataset
testdataset = pd.read_csv('/content/testing.csv')
#get only the temperature column
testdataset = testdataset.iloc[:30,1:2].values
real_temperature = pd.read_csv('/content/_testing.csv')
real_temperature = real_temperature.iloc[30:,1:2].values
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

In [ ]:
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))

1/1 [==============================] - 3s 3s/step


In [ ]:
real_temperature


array([[81. ],
       [82. ],
       [80.5],
       [79. ],
       [78. ]])

In [ ]:
predicted_temperature

array([[79.28368],
       [78.75407],
       [78.41582],
       [78.21471]], dtype=float32)